# Day10_1: A/B 테스트 분석 (A/B Test Analysis) - 정답

## 🎯 실습 퀴즈 정답

---

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.stats.proportion import proportions_ztest, proportion_confint, confint_proportions_2indep
import math

print("✅ 라이브러리 임포트 완료")

---

### Q1. 전환율 계산하기 ⭐

**문제**: 다음 데이터에서 A그룹과 B그룹의 전환율을 계산하세요.

In [ ]:
# Q1 정답

# 방문자 수
visitors_a = 500
visitors_b = 500

# 전환 수
conversions_a = 45
conversions_b = 60

# 1. 각 그룹 전환율 계산
cr_a = conversions_a / visitors_a * 100
cr_b = conversions_b / visitors_b * 100

# 2. 차이 계산
diff = cr_b - cr_a

# 3. 결과 출력
print("전환율 계산 결과")
print("=" * 40)
print(f"A그룹 전환율: {cr_a:.1f}%")
print(f"B그룹 전환율: {cr_b:.1f}%")
print(f"차이: {diff:.1f}%p")

In [ ]:
# 테스트 검증
assert cr_a == 9.0, "A그룹 전환율 계산 오류"
assert cr_b == 12.0, "B그룹 전환율 계산 오류"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
전환율은 전환 수를 방문자 수로 나누고 100을 곡하여 백분율로 표시합니다.

**핵심 개념**:
- 전환율 = (전환 수 / 방문자 수) x 100
- 절대적 차이는 %p(퍼센트 포인트)로 표시

**대안 솔루션**:
```python
# pandas를 활용한 방법
df = pd.DataFrame({
    'group': ['A', 'B'],
    'visitors': [500, 500],
    'conversions': [45, 60]
})
df['conversion_rate'] = df['conversions'] / df['visitors'] * 100
```

**흔한 실수**:
- ❌ 100을 곡하지 않고 비율로만 표시
- ❌ 방문자와 전환 수를 반대로 나눔

**실무 팁**:
전환율 차이를 말할 때는 항상 %p(퍼센트 포인트)를 사용하세요. "3% 증가"는 상대적 증가와 혼동될 수 있습니다.

---

### Q2. 그룹별 기초 통계량 계산 ⭐

**문제**: 이메일 A/B 테스트 데이터를 로드하여 그룹별 오픈율과 클릭율을 계산하세요.

In [ ]:
# Q2 정답

# 1. 데이터 로드
email_df = pd.read_csv('datasets/ab_test_email_subject.csv')

# 2. 그룹별 통계량 계산
email_stats = email_df.groupby('group').agg({
    'sent': 'sum',       # 발송 수
    'opened': 'sum',     # 오픈 수
    'clicked': 'sum'     # 클릭 수
}).reset_index()

# 3. 오픈율 계산 (오픈/발송)
email_stats['open_rate'] = email_stats['opened'] / email_stats['sent'] * 100

# 4. 클릭율 계산 (클릭/오픈) - CTR
email_stats['click_rate'] = email_stats['clicked'] / email_stats['opened'] * 100

# 5. 결과 출력
print("이메일 A/B 테스트 그룹별 통계량")
print("=" * 60)
for _, row in email_stats.iterrows():
    print(f"\n{row['group']}그룹:")
    print(f"  발송: {row['sent']}통")
    print(f"  오픈: {row['opened']}통")
    print(f"  클릭: {row['clicked']}통")
    print(f"  오픈율: {row['open_rate']:.1f}%")
    print(f"  클릭율(CTR): {row['click_rate']:.1f}%")

In [ ]:
# 테스트 검증
assert len(email_stats) == 2, "그룹이 2개여야 합니다"
assert 'open_rate' in email_stats.columns, "오픈율 칼럼이 필요합니다"
assert 'click_rate' in email_stats.columns, "클릭율 칼럼이 필요합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`groupby`로 그룹별로 집계한 후 비율을 계산합니다.

**핵심 개념**:
- 오픈율 = 오픈 수 / 발송 수 x 100
- 클릭율(CTR) = 클릭 수 / 오픈 수 x 100 (오픈한 사람 중 클릭 비율)

**대안 솔루션**:
```python
# 한 번에 계산하는 방법
email_stats = email_df.groupby('group').apply(
    lambda x: pd.Series({
        'open_rate': x['opened'].sum() / x['sent'].sum() * 100,
        'click_rate': x['clicked'].sum() / x['opened'].sum() * 100
    })
)
```

**흔한 실수**:
- ❌ 클릭율을 발송 대비로 계산 (오픈 대비가 정확)
- ❌ groupby 후 reset_index() 누락

**실무 팁**:
이메일 마케팅에서 CTR은 보통 "오픈한 사람 중 클릭 비율"을 의미합니다. 이를 CTOR(Click-to-Open Rate)이라고도 부릅니다.

---

### Q3. Plotly 바 차트로 전환율 비교 ⭐⭐

**문제**: 다음 데이터로 Plotly 바 차트를 생성하세요.

In [ ]:
# Q3 정답

# 1. 데이터 준비
campaign_data = pd.DataFrame({
    '캔페인': ['A (10% 할인)', 'B (20% 할인)', 'C (1+1 행사)'],
    '전환율': [3.2, 4.5, 5.1]
})

# 2. 바 차트 생성
fig = px.bar(
    campaign_data,
    x='캔페인',
    y='전환율',
    color='캔페인',
    text=campaign_data['전환율'].apply(lambda x: f'{x}%'),  # 막대 위 텍스트
    title='캔페인별 전환율 비교',
    width=600,
    height=450
)

# 3. 스타일 설정
fig.update_traces(textposition='outside', textfont_size=14)
fig.update_layout(
    yaxis_title='전환율 (%)',
    xaxis_title='',
    showlegend=False,
    font=dict(size=12)
)

fig.show()

In [ ]:
# 테스트 검증
assert fig is not None, "차트가 생성되어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`px.bar()`를 사용하여 기본 바 차트를 생성하고, `text` 파라미터로 막대 위에 값을 표시합니다.

**핵심 개념**:
- `text`: 막대 위/안에 표시할 텍스트
- `update_traces(textposition='outside')`: 막대 바깥에 표시
- `color`: 고유한 색상 할당

**대안 솔루션**:
```python
# go.Bar 사용
fig = go.Figure(go.Bar(
    x=campaign_data['캔페인'],
    y=campaign_data['전환율'],
    text=[f'{v}%' for v in campaign_data['전환율']],
    textposition='outside'
))
```

**흔한 실수**:
- ❌ `text` 파라미터 누락
- ❌ `textposition` 설정 누락 (inside가 기본값)

**실무 팁**:
A/B 테스트 결과 보고 시 바 차트에 신뢰구간을 함께 표시하면 더 설득력 있습니다.

---

### Q4. t-test로 전환율 차이 검정 ⭐⭐

**문제**: 다음 데이터로 두 그룹의 전환율 차이가 통계적으로 유의한지 검정하세요.

In [ ]:
# Q4 정답

# 각 사용자의 전환 여부 (0 또는 1)
group_a_conv = [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
group_b_conv = [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]

# 1. 각 그룹 전환율 확인
cr_a = sum(group_a_conv) / len(group_a_conv) * 100
cr_b = sum(group_b_conv) / len(group_b_conv) * 100

# 2. t-test 수행
t_stat, p_value = stats.ttest_ind(group_a_conv, group_b_conv)

# 3. 결과 출력
print("t-test 결과")
print("=" * 50)
print(f"\nA그룹 전환율: {cr_a:.1f}%")
print(f"B그룹 전환율: {cr_b:.1f}%")
print(f"\nt-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# 4. 결론
print("\n" + "=" * 50)
alpha = 0.05
if p_value < alpha:
    print(f"✅ p-value ({p_value:.4f}) < {alpha}")
    print("→ 결론: 두 그룹의 전환율 차이가 통계적으로 유의합니다!")
else:
    print(f"❌ p-value ({p_value:.4f}) >= {alpha}")
    print("→ 결론: 전환율 차이가 통계적으로 유의하지 않습니다.")

In [ ]:
# 테스트 검증
assert p_value < 0.05, "p-value가 0.05 미만이어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
0/1 값의 리스트로 독립 표본 t-test를 수행합니다. 0/1 데이터의 평균은 비율을 의미하므로 전환율 비교가 가능합니다.

**핵심 개념**:
- `stats.ttest_ind()`: 두 독립된 그룹의 평균 비교
- t-statistic: 음수면 A > B, 양수면 B > A
- p-value < 0.05면 통계적으로 유의한 차이

**대안 솔루션**:
```python
# 비율 데이터에는 z-test가 더 적합
from statsmodels.stats.proportion import proportions_ztest
count = [sum(group_a_conv), sum(group_b_conv)]
nobs = [len(group_a_conv), len(group_b_conv)]
z_stat, p_value = proportions_ztest(count, nobs)
```

**흔한 실수**:
- ❌ `ttest_rel`(대응 표본) 사용 - 독립 표본에는 `ttest_ind`
- ❌ p-value와 유의수준 비교 방향 혹동

**실무 팁**:
전환율 같은 비율 데이터는 t-test보다 z-test가 더 정확합니다. 샘플 크기가 크면 두 결과는 비슷합니다.

---

### Q5. p-value 해석 ⭐⭐

**문제**: 다음 p-value 값들에 대해 귀무가설 기각 여부를 판단하고 해석하세요.

In [ ]:
# Q5 정답

p_values = [0.001, 0.03, 0.05, 0.12, 0.48]
alpha = 0.05

print("p-value 해석")
print("=" * 60)
print(f"유의수준 (α): {alpha}")
print()

for p in p_values:
    if p < alpha:
        decision = "✅ 귀무가설 기각 (통계적으로 유의함)"
    elif p == alpha:
        decision = "⚠️ 경계선 (p = α, 신중한 해석 필요)"
    else:
        decision = "❌ 귀무가설 채택 (통계적으로 유의하지 않음)"
    
    print(f"p-value = {p:.3f} → {decision}")

In [ ]:
# 테스트 검증
rejected = [p for p in p_values if p < alpha]
assert len(rejected) == 2, "기각되는 p-value는 2개여야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
p-value와 유의수준(alpha)을 비교하여 귀무가설 기각 여부를 판단합니다.

**핵심 개념**:
- p < α: 귀무가설 기각 (유의한 차이 있음)
- p >= α: 귀무가설 채택 (유의한 차이 없음)
- p = α: 경계선, 신중한 해석 필요

**대안 솔루션**:
```python
# 리스트 컴프리헨션으로 한 번에
results = ['Reject' if p < 0.05 else 'Accept' for p in p_values]
```

**흔한 실수**:
- ❌ p < 0.05를 "확실한 차이"로 해석 (95% 신뢰도일 뿐)
- ❌ p > 0.05를 "차이가 없다"로 해석 ("차이를 발견하지 못했다"가 정확)

**실무 팁**:
p = 0.05인 경우 확실한 결론을 내리기 어렵습니다. 샘플을 늘리거나 효과 크기를 함께 고려하세요.

---

---

## Part 2 심화 퀴즈 정답

---

### Q6. 비율 검정 (z-test) 수행 ⭐⭐⭐

**문제**: 다음 데이터로 z-test를 수행하여 두 그룹의 전환율 차이를 검정하세요.

In [ ]:
# Q6 정답

from statsmodels.stats.proportion import proportions_ztest

# 각 그룹 전환 수와 총 샘플
conversions = np.array([85, 105])  # A그룹 85건, B그룹 105건 전환
samples = np.array([1000, 1000])   # 각 1000명씩

# 1. 전환율 계산
cr_a = conversions[0] / samples[0] * 100
cr_b = conversions[1] / samples[1] * 100

# 2. z-test 수행
z_stat, p_value = proportions_ztest(conversions, samples, alternative='two-sided')

# 3. 결과 출력
print("비율 검정 (z-test) 결과")
print("=" * 50)
print(f"\nA그룹 전환율: {cr_a:.1f}%")
print(f"B그룹 전환율: {cr_b:.1f}%")
print(f"차이: {cr_b - cr_a:.1f}%p")
print(f"\nz-statistic: {z_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# 4. 결론
print("\n" + "=" * 50)
if p_value < 0.05:
    print("✅ 두 그룹의 전환율 차이가 통계적으로 유의합니다!")
else:
    print("❌ 전환율 차이가 통계적으로 유의하지 않습니다.")

In [ ]:
# 테스트 검증
assert abs(z_stat) > 0, "z-statistic이 계산되어야 합니다"
assert 0 < p_value < 1, "p-value는 0과 1 사이여야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`proportions_ztest`는 두 그룹의 비율(전환율) 차이를 검정하는 데 적합한 통계 방법입니다.

**핵심 개념**:
- `proportions_ztest(count, nobs)`: 각 그룹의 성공 수와 전체 수를 입력
- `alternative='two-sided'`: 양측 검정 (A != B)
- `alternative='larger'`: B > A 검정

**대안 솔루션**:
```python
# Chi-square 검정으로도 가능
contingency_table = [[85, 915], [105, 895]]  # [전환, 비전환]
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
```

**흔한 실수**:
- ❌ count와 nobs 순서 혹동
- ❌ 비율 대신 평균 값 입력

**실무 팁**:
A/B 테스트에서는 t-test보다 z-test가 비율 데이터에 더 적합합니다. 샘플 크기가 30 이상이면 둘의 결과는 비슷합니다.

---

### Q7. 신뢰구간 계산하기 ⭐⭐⭐⭐

**문제**: 전환율 12% (120/1000)의 95% 신뢰구간을 계산하세요.

In [ ]:
# Q7 정답

from statsmodels.stats.proportion import proportion_confint

conversions = 120
total = 1000

# 1. 전환율 계산
conversion_rate = conversions / total * 100

# 2. 95% 신뢰구간 계산 (Wilson 방법 권장)
ci_lower, ci_upper = proportion_confint(
    count=conversions,
    nobs=total,
    alpha=0.05,
    method='wilson'
)

# 3. 결과 출력
print("전환율 95% 신뢰구간")
print("=" * 50)
print(f"\n전환 수: {conversions}")
print(f"총 샘플: {total}")
print(f"\n전환율: {conversion_rate:.2f}%")
print(f"95% 신뢰구간: [{ci_lower*100:.2f}%, {ci_upper*100:.2f}%]")
print(f"\n해석: 실제 전환율은 {ci_lower*100:.2f}%와 {ci_upper*100:.2f}% 사이일 것으로 예상")

In [ ]:
# 테스트 검증
assert ci_lower < conversion_rate/100 < ci_upper, "신뢰구간이 전환율을 포함해야 합니다"
assert ci_lower > 0 and ci_upper < 1, "신뢰구간은 0~1 사이여야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`proportion_confint`를 사용하여 비율의 신뢰구간을 계산합니다. Wilson 방법이 소표본에서도 안정적입니다.

**핵심 개념**:
- `alpha=0.05`: 95% 신뢰구간 (1-0.05)
- `method='wilson'`: Wilson score interval (권장)
- `method='normal'`: Wald interval (대표본에 적합)

**대안 솔루션**:
```python
# 수동 계산 (Wald 방법)
import scipy.stats as stats
p = conversions / total
se = np.sqrt(p * (1-p) / total)
z = stats.norm.ppf(0.975)  # 1.96
ci_lower = p - z * se
ci_upper = p + z * se
```

**흔한 실수**:
- ❌ 비율을 백분율로 입력 (0~1로 입력해야 함)
- ❌ alpha를 0.95로 설정 (0.05가 정확)

**실무 팁**:
샘플 크기가 작으면 Wilson 방법, 크면 Normal(클로퍼-피어슨)이나 Wald 방법을 사용하세요.

---

### Q8. 신뢰구간 시각화 ⭐⭐⭐⭐

**문제**: 3개 캔페인의 전환율과 신뢰구간을 오차 바 차트로 시각화하세요.

In [ ]:
# Q8 정답

campaigns = pd.DataFrame({
    '캔페인': ['A', 'B', 'C'],
    '전환율': [4.5, 5.2, 4.8],
    'CI_lower': [3.8, 4.6, 4.1],
    'CI_upper': [5.2, 5.8, 5.5]
})

# 오차 계산 (error bar 용)
campaigns['error_plus'] = campaigns['CI_upper'] - campaigns['전환율']
campaigns['error_minus'] = campaigns['전환율'] - campaigns['CI_lower']

# 오차 바 차트 생성
fig = go.Figure()

# 각 캔페인별로 마커 추가
colors = ['#3498db', '#e74c3c', '#2ecc71']
for i, row in campaigns.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['캔페인']],
        y=[row['전환율']],
        error_y=dict(
            type='data',
            symmetric=False,
            array=[row['error_plus']],
            arrayminus=[row['error_minus']]
        ),
        mode='markers',
        marker=dict(size=15, color=colors[i]),
        name=f"캔페인 {row['캔페인']}"
    ))

fig.update_layout(
    title='캔페인별 전환율 95% 신뢰구간',
    yaxis_title='전환율 (%)',
    xaxis_title='',
    showlegend=True,
    width=600,
    height=450,
    font=dict(size=12)
)

fig.show()

In [ ]:
# 테스트 검증
assert fig is not None, "차트가 생성되어야 합니다"
assert len(fig.data) == 3, "캔페인 3개의 마커가 필요합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`go.Scatter`의 `error_y` 파라미터를 사용하여 오차 바를 표시합니다. 비대칭 오차를 위해 `symmetric=False`를 설정합니다.

**핵심 개념**:
- `error_y`: 오차 바 설정
- `symmetric=False`: 상하 독립 오차 설정
- `array`: 위쪽 오차 (평균에서 상한까지)
- `arrayminus`: 아래쪽 오차 (평균에서 하한까지)

**대안 솔루션**:
```python
# 한 번에 모든 마커 추가
fig = go.Figure(go.Scatter(
    x=campaigns['캔페인'],
    y=campaigns['전환율'],
    error_y=dict(
        type='data',
        symmetric=False,
        array=campaigns['error_plus'],
        arrayminus=campaigns['error_minus']
    ),
    mode='markers',
    marker=dict(size=15)
))
```

**흔한 실수**:
- ❌ 오차 값 계산 오류 (CI - 평균이 아닌 CI 값 자체 입력)
- ❌ `symmetric=True`로 비대칭 신뢰구간 표시

**실무 팁**:
신뢰구간이 겹치면 통계적으로 유의한 차이가 없을 가능성이 높습니다. 시각적으로 불확실성을 전달하기 좋습니다.

---

### Q9. Cohen's h 효과 크기 계산 ⭐⭐⭐⭐⭐

**문제**: 다음 두 그룹의 전환율 차이에 대한 Cohen's h를 계산하고 해석하세요.

In [ ]:
# Q9 정답

import math

p_control = 0.08    # 통제그룹 전환율 8%
p_treatment = 0.12  # 실험그룹 전환율 12%

# 1. Cohen's h 계산 함수
def cohens_h(p1, p2):
    """
    Cohen's h 효과 크기 계산
    phi = 2 * arcsin(sqrt(p))
    h = |phi1 - phi2|
    """
    phi1 = 2 * math.asin(math.sqrt(p1))
    phi2 = 2 * math.asin(math.sqrt(p2))
    return abs(phi1 - phi2)

# 2. 계산
h = cohens_h(p_control, p_treatment)

# 3. 해석
if h < 0.2:
    effect_size = "작은 효과 (Small)"
elif h < 0.5:
    effect_size = "중간 효과 (Medium)"
else:
    effect_size = "큰 효과 (Large)"

# 4. 결과 출력
print("Cohen's h 효과 크기 분석")
print("=" * 50)
print(f"\n통제그룹 전환율: {p_control*100:.1f}%")
print(f"실험그룹 전환율: {p_treatment*100:.1f}%")
print(f"차이: {(p_treatment - p_control)*100:.1f}%p")
print(f"\nCohen's h: {h:.4f}")
print(f"해석: {effect_size}")
print("\n해석 기준:")
print("  |h| < 0.2  : 작은 효과")
print("  0.2 ≤ |h| < 0.5: 중간 효과")
print("  |h| ≥ 0.5 : 큰 효과")

In [ ]:
# 테스트 검증
assert h > 0, "Cohen's h는 양수여야 합니다"
assert 0.1 < h < 0.3, "h는 대략 0.13 정도여야 합니다 (작은 효과)"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
Cohen's h는 두 비율의 차이를 표준화하여 효과 크기를 측정합니다. arcsin 변환을 사용합니다.

**핵심 개념**:
- Cohen's h = |2*arcsin(√p1) - 2*arcsin(√p2)|
- 작은 효과: h < 0.2
- 중간 효과: 0.2 ≤ h < 0.5
- 큰 효과: h ≥ 0.5

**대안 솔루션**:
```python
# statsmodels 사용
from statsmodels.stats.proportion import proportion_effectsize
h = proportion_effectsize(p_treatment, p_control)
```

**흔한 실수**:
- ❌ 단순 비율 차이를 효과 크기로 오해 (4%p != 0.04 effect)
- ❌ 라디안이 아닌 도(degree)로 계산

**실무 팁**:
통계적 유의성(p-value)과 효과 크기(Cohen's h)는 별개입니다. 샘플이 크면 작은 차이도 유의할 수 있지만, 실무적으로 의미 있는 차이인지는 효과 크기로 판단하세요.

---

### Q10. 종합 A/B 테스트 분석 리포트 ⭐⭐⭐⭐⭐

**문제**: 이메일 마케팅 A/B 테스트 데이터를 완전히 분석하세요.

In [ ]:
# Q10 정답

# 1. 데이터 로드
email_df = pd.read_csv('datasets/ab_test_email_subject.csv')

print("=" * 70)
print("          이메일 A/B 테스트 종합 분석 리포트")
print("=" * 70)

In [ ]:
# 2. 기초 통계량 계산
print("\n📊 1. 기초 통계량")
print("-" * 70)

# 그룹별 집계
group_stats = email_df.groupby('group').agg({
    'sent': 'sum',
    'opened': 'sum',
    'clicked': 'sum'
}).reset_index()

# 오픈율, 클릭율 계산
group_stats['open_rate'] = group_stats['opened'] / group_stats['sent'] * 100
group_stats['click_rate'] = group_stats['clicked'] / group_stats['opened'] * 100

for _, row in group_stats.iterrows():
    print(f"\n{row['group']}그룹:")
    print(f"  발송: {row['sent']}통")
    print(f"  오픈: {row['opened']}통 (Open Rate: {row['open_rate']:.1f}%)")
    print(f"  클릭: {row['clicked']}통 (CTR: {row['click_rate']:.1f}%)")

In [ ]:
# 3. z-test로 오픈율 차이 검정
print("\n🔬 2. 통계적 검정 (z-test)")
print("-" * 70)

# 데이터 추출
a_stats = group_stats[group_stats['group'] == 'A'].iloc[0]
b_stats = group_stats[group_stats['group'] == 'B'].iloc[0]

# 오픈율 z-test
count_open = np.array([a_stats['opened'], b_stats['opened']])
nobs_open = np.array([a_stats['sent'], b_stats['sent']])
z_stat_open, p_value_open = proportions_ztest(count_open, nobs_open)

print(f"\n오픈율 검정:")
print(f"  A그룹: {a_stats['open_rate']:.1f}%")
print(f"  B그룹: {b_stats['open_rate']:.1f}%")
print(f"  z-statistic: {z_stat_open:.4f}")
print(f"  p-value: {p_value_open:.4f}")

if p_value_open < 0.05:
    print(f"  ✅ 통계적으로 유의함 (p < 0.05)")
else:
    print(f"  ❌ 통계적으로 유의하지 않음 (p >= 0.05)")

In [ ]:
# 4. 95% 신뢰구간 계산
print("\n📊 3. 95% 신뢰구간")
print("-" * 70)

ci_a = proportion_confint(a_stats['opened'], a_stats['sent'], alpha=0.05, method='wilson')
ci_b = proportion_confint(b_stats['opened'], b_stats['sent'], alpha=0.05, method='wilson')

print(f"\nA그룹 오픈율: {a_stats['open_rate']:.1f}% [{ci_a[0]*100:.1f}%, {ci_a[1]*100:.1f}%]")
print(f"B그룹 오픈율: {b_stats['open_rate']:.1f}% [{ci_b[0]*100:.1f}%, {ci_b[1]*100:.1f}%]")

# 신뢰구간 겹침 여부
if ci_a[1] < ci_b[0]:
    print("✅ 신뢰구간이 겹치지 않음 - B그룹이 확실히 높음")
elif ci_b[1] < ci_a[0]:
    print("✅ 신뢰구간이 겹치지 않음 - A그룹이 확실히 높음")
else:
    print("⚠️ 신뢰구간이 겹침 - 차이의 확실성이 낮음")

In [ ]:
# 5. Cohen's h 효과 크기
print("\n📈 4. 효과 크기 (Cohen's h)")
print("-" * 70)

def cohens_h(p1, p2):
    phi1 = 2 * math.asin(math.sqrt(p1))
    phi2 = 2 * math.asin(math.sqrt(p2))
    return abs(phi1 - phi2)

p_a = a_stats['opened'] / a_stats['sent']
p_b = b_stats['opened'] / b_stats['sent']
h = cohens_h(p_a, p_b)

if h < 0.2:
    effect = "작은 효과 (Small)"
elif h < 0.5:
    effect = "중간 효과 (Medium)"
else:
    effect = "큰 효과 (Large)"

print(f"\nCohen's h: {h:.4f}")
print(f"해석: {effect}")

In [ ]:
# 6. 세그먼트별 분석
print("\n👥 5. 세그먼트별 분석 (user_segment)")
print("-" * 70)

segment_stats = email_df.groupby(['user_segment', 'group']).agg({
    'sent': 'sum',
    'opened': 'sum'
}).reset_index()
segment_stats['open_rate'] = segment_stats['opened'] / segment_stats['sent'] * 100

segment_pivot = segment_stats.pivot(
    index='user_segment', 
    columns='group', 
    values='open_rate'
).round(1)
segment_pivot['diff'] = segment_pivot['B'] - segment_pivot['A']

print("\n세그먼트별 오픈율:")
print(segment_pivot)

In [ ]:
# 7. 최종 권고사항
print("\n" + "=" * 70)
print("🎯 최종 권고사항")
print("=" * 70)

# 조건 확인
is_significant = p_value_open < 0.05
is_b_better = b_stats['open_rate'] > a_stats['open_rate']
is_meaningful = h >= 0.2

if is_significant and is_b_better:
    print("\n✅ B그룹(신규 제목)을 전체 사용자에게 적용하시기 바랍니다.")
    print(f"   - 통계적으로 유의한 오픈율 개선 확인 (p = {p_value_open:.4f})")
    print(f"   - 오픈율 {b_stats['open_rate'] - a_stats['open_rate']:.1f}%p 향상")
    if is_meaningful:
        print(f"   - 실무적으로도 의미 있는 효과 (h = {h:.4f})")
    else:
        print(f"   - 단, 효과 크기는 작음 (h = {h:.4f})")
else:
    print("\n⚠️ 추가 테스트가 필요합니다.")
    if not is_significant:
        print(f"   - 통계적 유의성 부족 (p = {p_value_open:.4f})")
    print("   - 샘플 크기를 늘리거나 다른 제목을 테스트하세요")

print("\n" + "=" * 70)

In [ ]:
# 테스트 검증
assert 'open_rate' in group_stats.columns, "오픈율 계산 필요"
assert h is not None, "Cohen's h 계산 필요"
assert len(segment_pivot) > 0, "세그먼트 분석 필요"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
A/B 테스트 분석의 전체 프로세스를 단계별로 수행합니다:
1. 기초 통계량 계산
2. 통계적 검정 (z-test)
3. 신뢰구간 계산
4. 효과 크기 측정
5. 세그먼트별 분석
6. 최종 의사결정

**핵심 개념**:
- 전체 결과와 세그먼트별 결과가 다를 수 있음 (Simpson's Paradox)
- p-value, 신뢰구간, 효과 크기를 종합적으로 판단
- 통계적 유의성과 실무적 중요성을 모두 고려

**대안 솔루션**:
```python
# 베이지안 A/B 테스트 (추후 학습)
# Frequentist 방법의 한계를 보완
from scipy import stats
# Beta 분포를 사용한 확률적 해석
```

**흔한 실수**:
- ❌ p-value만으로 의사결정 (effect size 무시)
- ❌ 전체 결과만 보고 세그먼트 분석 누락
- ❌ 비즈니스 임팩트 고려 없이 결정

**실무 팁**:
1. 항상 샘플 크기와 실험 기간을 먼저 확인하세요
2. 전체 결과와 함께 세그먼트별 결과도 확인하세요
3. 통계적 유의성, 신뢰구간, 효과 크기를 종합적으로 판단하세요
4. 최종 의사결정은 비즈니스 임팩트를 기준으로 하세요

---

## 📊 학습 정리 - 정답 요약

| 퀴즈 | 핵심 개념 | 주요 함수/방법 |
|------|----------|----------------|
| Q1 | 전환율 계산 | `conversions / visitors * 100` |
| Q2 | 그룹별 통계 | `df.groupby().agg()` |
| Q3 | 바 차트 | `px.bar(text=, textposition=)` |
| Q4 | t-test | `stats.ttest_ind()` |
| Q5 | p-value 해석 | `p < 0.05 → 귀무가설 기각` |
| Q6 | z-test | `proportions_ztest(count, nobs)` |
| Q7 | 신뢰구간 | `proportion_confint()` |
| Q8 | 오차 바 차트 | `go.Scatter(error_y=)` |
| Q9 | Cohen's h | `2*arcsin(sqrt(p))` 차이 |
| Q10 | 종합 분석 | 위 모든 개념 통합 |